In [1]:
import numpy as np

In [2]:
from scipy.stats import norm
import matplotlib.pyplot as plt

In [3]:
def plot_distribution(ax, pdfs, title=''): 
    x = np.linspace(0., 10., 200)
    ymax = 0    
    for index, pdf in enumerate(pdfs):
        y = norm.pdf(x, pdf.mean, np.sqrt(pdf.var))

        p = ax.plot(x, y, lw = 2)
        c = p[0].get_markeredgecolor()    
        ax.fill_between(x, y, 0, color=c, alpha=0.2 )    
        ax.autoscale(tight=True)
        ax.vlines(pdf.mean, 0, y.max(), colors = c, linestyles = "--", lw = 2)    

        ymax = max( ymax, y[1:].max()*1.05 )
    ax.set_ylim([0,ymax])

def plot_regret(ax, regret, title=''): 
    ax.plot(regret)

def plot(regret, mab):
    fig, axs = plt.subplots(1, 2, figsize=(6, 3))
    plot_regret(axs[0], regret)
    plot_distribution(axs[1], mab)

In [4]:
class Arm:
    def __init__(self, mean, var):
        self.mean = mean
        self.var = var

    def sample(self):
        return np.random.normal(self.mean, np.sqrt(self.var))

    def name(self):
        return 'N(' + str(self.mean) + ',' + str(self.var) + ')'

# Thompson Sampling

In this exercise we will run thompson sampling for 2-armed bandit with gaussian distribution.
For simplicity assume we know variance of distribution of arms and only mean is unknown
for prior assume gaussian distribution.

**Answer:**

If the variance is unknown, we would use a **Normal-Inverse-Gamma (NIG)** distribution as the conjugate prior for the Gaussian likelihood with unknown mean and variance.

Alternatively, we could use:
- **Inverse-Gamma** distribution for the variance alone
- **Gamma** distribution for the precision (inverse of variance)
- **Student's t-distribution** as a marginal distribution when integrating out the variance

The Normal-Inverse-Gamma is the most common choice because it's the conjugate prior for Gaussian data with both unknown mean and variance, which makes Bayesian updates analytically tractable.


## 1.
if variance was unknown what prior distribtion would be suitable?

## 2.
Implement Thompson Sampling algorithm. For comparison also implement ϵ-Greedy and UCB algorithms

**Implementation Overview:**

We implement three classic bandit algorithms:

1. **Thompson Sampling**: Bayesian approach that samples from posterior distributions
   - Naturally balances exploration and exploitation
   - Probabilistic action selection based on uncertainty
   
2. **Upper Confidence Bound (UCB)**: Deterministic approach using confidence intervals
   - Selects arm with highest upper confidence bound
   - Theoretical regret guarantees
   
3. **ε-Greedy**: Simple approach with random exploration
   - Explores randomly with probability ε
   - Exploits best known arm with probability (1-ε)

All three maintain value estimates Q(a) and update them incrementally using: Q(a) ← Q(a) + α[R - Q(a)]


In [ ]:
class ThompsonSampling:
    def __init__(self, var_list, **kwargs):
        """
        variance of arms are known to policy
        """
        self.var_list = var_list
        self.n_arms = len(var_list)
        
        # Initialize prior beliefs: mean and variance for each arm
        self.prior_means = np.zeros(self.n_arms)
        self.prior_vars = np.ones(self.n_arms) * 100  # Large initial variance (high uncertainty)
        
        # Track number of pulls for each arm
        self.n_pulls = np.zeros(self.n_arms)

    def select_arm(self, *args):
        # ==================================== Your Code (Begin) ==================================
        
        # Sample from the posterior distribution of each arm
        samples = []
        for i in range(self.n_arms):
            # Sample from N(prior_mean, prior_var)
            sample = np.random.normal(self.prior_means[i], np.sqrt(self.prior_vars[i]))
            samples.append(sample)
        
        # Select arm with highest sample (optimistic estimate)
        selected_arm = np.argmax(samples)
        
        # return index of selected arm
        return selected_arm

        # ==================================== Your Code (End) ====================================

    def update(self, idx, reward):
        # ==================================== Your Code (Begin) ==================================
        
        # Bayesian update for Gaussian with known variance
        # Posterior is also Gaussian with updated mean and variance
        
        # Prior: N(μ₀, σ₀²)
        # Likelihood: N(reward, σ²) where σ² is the known arm variance
        # Posterior: N(μ₁, σ₁²)
        
        prior_mean = self.prior_means[idx]
        prior_var = self.prior_vars[idx]
        obs_var = self.var_list[idx]
        
        # Bayesian update formulas
        posterior_var = 1.0 / (1.0/prior_var + 1.0/obs_var)
        posterior_mean = posterior_var * (prior_mean/prior_var + reward/obs_var)
        
        # Update beliefs
        self.prior_means[idx] = posterior_mean
        self.prior_vars[idx] = posterior_var
        self.n_pulls[idx] += 1
        
        # ==================================== Your Code (End) ====================================

In [ ]:
class UCB:    
    def __init__(self, n_bandits, c_level):
        """
        c_level: coefficient of uncertainty
        """
        self.n_bandits = n_bandits
        self.c_level = c_level
        
        # Track estimates and counts
        self.Q = np.zeros(n_bandits)  # Estimated values
        self.N = np.zeros(n_bandits)  # Number of times each arm was pulled
    
    def select_arm(self, t):
        """
        t: step time
        """
        # ==================================== Your Code (Begin) ==================================
        
        # Initially pull each arm once to avoid division by zero
        for i in range(self.n_bandits):
            if self.N[i] == 0:
                return i
        
        # Calculate UCB values for each arm
        ucb_values = self.Q + self.c_level * np.sqrt(np.log(t + 1) / self.N)
        
        # return index of selected arm
        return np.argmax(ucb_values)

        # ==================================== Your Code (End) ====================================

    def update(self, idx, reward):
        # ==================================== Your Code (Begin) ==================================
        
        # Incremental update of mean estimate
        self.N[idx] += 1
        self.Q[idx] = self.Q[idx] + (reward - self.Q[idx]) / self.N[idx]
        
        # ==================================== Your Code (End) ====================================

In [ ]:
class eGreedy:    
    def __init__(self, n_bandits, epsilon):
        """
        epsilon must be given
        """
        self.n_bandits = n_bandits
        self.epsilon = epsilon
        
        # Track estimates and counts
        self.Q = np.zeros(n_bandits)  # Estimated values
        self.N = np.zeros(n_bandits)  # Number of times each arm was pulled
    
    def select_arm(self, *args):
        # ==================================== Your Code (Begin) ==================================
        
        # Explore with probability epsilon
        if np.random.random() < self.epsilon:
            # Explore: select random arm
            return np.random.randint(0, self.n_bandits)
        else:
            # Exploit: select best arm based on current estimates
            return np.argmax(self.Q)

        # ==================================== Your Code (End) ====================================

    def update(self, idx, reward):
        # ==================================== Your Code (Begin) ==================================
        
        # Incremental update of mean estimate
        self.N[idx] += 1
        self.Q[idx] = self.Q[idx] + (reward - self.Q[idx]) / self.N[idx]
        
        # ==================================== Your Code (End) ====================================

## 3
run simulation for arms described as cells below and describe the differences of regret with different variance in arms distributions

rum_sim1 must return cumulitive regret formulated as 
$$
R(T)=\sum_{i=1}^2 N_i(T) \Delta_i
$$

where $N_i(T)$ is number of times arm $i$ was selected until step $T$, $\Delta_i=\mu^*-\mu_i$, $\mu^*$ is largest mean in arms distribtions and $\mu_i$ is mean of distribution of arm $i$

to get average regret we rum simulation 50 times.

In [ ]:
def run_sim1(policy, mab, step_num=100):
    """
    run simulation of multi-armed bandit
    mab: list of arms
    """
    best_mean = np.max([b.mean for b in mab])
    regret = []
    cumulative_regret = 0
    
    for k in range(step_num):          
        # ==================================== Your Code (Begin) ==================================
        
        # Select arm using policy
        arm_idx = policy.select_arm(k)
        
        # Sample reward from selected arm
        reward = mab[arm_idx].sample()
        
        # Update policy with observed reward
        policy.update(arm_idx, reward)
        
        # Calculate instantaneous regret
        instantaneous_regret = best_mean - mab[arm_idx].mean
        cumulative_regret += instantaneous_regret
        
        # Store cumulative regret
        regret.append(cumulative_regret)
 
        # ==================================== Your Code (End) ====================================
    
    return regret

**Understanding Regret:**

Regret measures how much reward we lose compared to always selecting the optimal arm.

**Cumulative Regret Formula:** R(T) = Σᵢ Nᵢ(T) × Δᵢ

Where:
- **Nᵢ(T)**: Number of times arm i was selected up to time T
- **Δᵢ = μ* - μᵢ**: Gap between optimal mean μ* and arm i's mean
- **μ***: Maximum mean among all arms

**Interpretation:**
- Perfect algorithm: Always picks optimal arm → R(T) = 0
- Random selection: R(T) grows linearly with T
- Good algorithms: R(T) grows sublinearly (e.g., logarithmic for UCB)

**In our plots:**
- Y-axis shows cumulative regret over time
- Flatter curves = better performance (less regret)
- Initial exploration causes regret growth, then should plateau as optimal arm is identified


### 3.1
Assume Multi-Armed Bandit variables are as follows.

In [9]:
mab = [Arm(6, 0.5), Arm(4, 0.5)]

**Thompson Sampling Explanation:**

Thompson Sampling uses Bayesian inference to balance exploration and exploitation:
1. We maintain a posterior distribution over each arm's mean reward
2. At each step, we sample from each arm's posterior distribution
3. We select the arm with the highest sample (optimistic action)
4. After observing the reward, we update the posterior using Bayes' rule

With low variance (0.5), the arms are more predictable. Thompson Sampling should converge quickly to the optimal arm, and the regret plot should show initial growth that plateaus.


#### Thompson Sampling
run and describe the result.

**ε-Greedy Explanation:**

ε-Greedy is a simple exploration strategy:
- With probability ε, explore by selecting a random arm
- With probability (1-ε), exploit by selecting the arm with highest estimated value

The trade-off:
- **Small ε (e.g., 0.01)**: More exploitation, faster convergence but may get stuck in suboptimal arm
- **Large ε (e.g., 0.2)**: More exploration, slower to converge but less likely to miss the optimal arm
- Unlike Thompson Sampling, ε-Greedy explores uniformly without considering uncertainty


In [ ]:
regret = [run_sim1(ThompsonSampling([b.var for b in mab]), mab) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

#### ϵ-Greedy
run for different values of ϵ and compare results.

**UCB (Upper Confidence Bound) Explanation:**

UCB uses the principle of "optimism in the face of uncertainty":
- For each arm, compute: UCB(a) = Q(a) + c × √(ln(t) / N(a))
  - Q(a): estimated mean reward
  - N(a): number of times arm was pulled
  - t: total time steps
  - c: confidence level parameter

The algorithm always selects the arm with highest UCB value. The confidence term decreases with more observations, reducing exploration over time.

**Parameter c:**
- **Small c (e.g., 0.5)**: Less exploration, may converge faster but risk missing optimal arm
- **Large c (e.g., 3.0)**: More exploration, guaranteed to find optimal but slower convergence
- **c=2**: Often works well in practice, providing theoretical regret bounds


In [ ]:
epsilon = 0.1  # Try different values: 0.01, 0.05, 0.1, 0.2
regret = [run_sim1(eGreedy(2, epsilon=epsilon), mab) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

#### UCB
run for different values of confidence level and compare results.


In [ ]:
c_level = 2.0  # Try different values: 0.5, 1.0, 2.0, 3.0
regret = [run_sim1(UCB(2, c_level=c_level), mab) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

**Impact of High Variance:**

Now we test with variance = 10 (compared to 0.5 before). Key differences:
- **More noise**: Each reward sample is less informative about the true mean
- **Slower learning**: All algorithms need more samples to distinguish between arms
- **Higher regret**: The uncertainty makes it harder to identify the optimal arm quickly
- **Thompson Sampling advantage**: Should handle high variance better because it explicitly models uncertainty

With high variance, we run more steps (500 vs 100) to see convergence patterns more clearly.


### 3.2
Assume Multi-Armed Bandit variables are as follows.

In [19]:
mab = [Arm(6, 10), Arm(4, 10)]

#### Thompson Sampling
run and compare results.

In [ ]:
regret = [run_sim1(ThompsonSampling([b.var for b in mab]), mab, step_num=500) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

#### ϵ-Greedy
run for different values of ϵ and compare results.

In [ ]:
epsilon = 0.1  # Try different values: 0.01, 0.05, 0.1, 0.2
regret = [run_sim1(eGreedy(2, epsilon=epsilon), mab, step_num=500) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

#### UCB
run for different values of confidence level and compare results.

**Non-Stationary Environment Results:**

In this simulation, the first arm's mean changes from 6 to 2 at step 100:
- **Before step 100**: Arm 1 is optimal (mean=6 vs 4)
- **After step 100**: Arm 2 becomes optimal (mean=4 vs 2)

**Expected behavior:**
- Standard Thompson Sampling will learn to prefer Arm 1 initially
- After the change at step 100, it should eventually detect the change and switch
- However, the posterior has accumulated strong belief that Arm 1 is best, making adaptation slow
- You should see regret accelerate after step 100 as the algorithm continues pulling the now-suboptimal Arm 1


In [ ]:
c_level = 2.0  # Try different values: 0.5, 1.0, 2.0, 3.0
regret = [run_sim1(UCB(2, c_level=c_level), mab, step_num=500) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

## 4
simulation below assumes a non-stationary multi-armed bandit. specifically in this simulation mean value of distribution of first arm changes in step 100. describe the result of thompson sampling.

**Improved Thompson Sampling for Non-Stationary Environments:**

To handle non-stationary environments, we use a **sliding window approach**:
- Keep only the most recent N observations (e.g., buffer_size=30)
- Discard old observations that may reflect outdated arm statistics
- Recompute posterior based only on recent data

**Benefits:**
- **Faster adaptation**: Old beliefs are "forgotten" as buffer fills with new data
- **Maintains exploration**: By resetting the posterior periodically, we remain open to changes
- **Trade-off**: Buffer size controls adaptation speed vs. statistical efficiency

**Expected results:**
- Should see faster recovery after the distribution change at step 100
- Regret growth should slow down more quickly compared to standard Thompson Sampling
- The algorithm effectively "forgets" that Arm 1 was previously optimal


In [ ]:
def run_sim2(ts, mab, step_num=200, change_step=100):
    init_mean = mab[0].mean
    best_mean = np.max([b.mean for b in mab])
    regret = []
    cumulative_regret = 0
    
    for i in range(step_num):
        if i == change_step:
            mab[0].mean = 2
            best_mean = np.max([b.mean for b in mab])
            
                  
        # ==================================== Your Code (Begin) ==================================
        
        # Select arm using Thompson Sampling
        arm_idx = ts.select_arm(i)
        
        # Sample reward from selected arm
        reward = mab[arm_idx].sample()
        
        # Update policy with observed reward
        ts.update(arm_idx, reward)
        
        # Calculate instantaneous regret
        instantaneous_regret = best_mean - mab[arm_idx].mean
        cumulative_regret += instantaneous_regret
        
        # Store cumulative regret
        regret.append(cumulative_regret)
        
        # ==================================== Your Code (End) ====================================
    
    mab[0].mean = init_mean
    return regret

In [ ]:
mab = [Arm(6, 0.5), Arm(4, 0.5)]
regret = [run_sim2(ThompsonSampling([b.var for b in mab]), mab) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)

### 4.1
change thompson sampling algorithm to improve results in non-stationary MAB.

In [ ]:
class NewThompsonSampling:
    def __init__(self, var_list, buffer_size=30, **kwargs):
        """
        Thompson Sampling with sliding window for non-stationary environments
        buffer_size: maximum number of recent observations to keep per arm
        """
        self.var_list = var_list
        self.n_arms = len(var_list)
        self.buffer_size = buffer_size
        
        # Keep sliding window of recent rewards for each arm
        self.reward_buffers = [[] for _ in range(self.n_arms)]
        
        # Initialize prior beliefs
        self.prior_means = np.zeros(self.n_arms)
        self.prior_vars = np.ones(self.n_arms) * 100

    def select_arm(self, *args):
        # ==================================== Your Code (Begin) ==================================
        
        # Sample from the posterior distribution of each arm
        samples = []
        for i in range(self.n_arms):
            # Sample from N(prior_mean, prior_var)
            sample = np.random.normal(self.prior_means[i], np.sqrt(self.prior_vars[i]))
            samples.append(sample)
        
        # Select arm with highest sample
        selected_arm = np.argmax(samples)

        # return index of selected arm
        return selected_arm

        # ==================================== Your Code (End) ====================================

    def update(self, idx, reward):
        # ==================================== Your Code (Begin) ==================================
        
        # Add reward to buffer
        self.reward_buffers[idx].append(reward)
        
        # Keep only recent rewards (sliding window)
        if len(self.reward_buffers[idx]) > self.buffer_size:
            self.reward_buffers[idx].pop(0)
        
        # Update posterior based on recent observations only
        if len(self.reward_buffers[idx]) > 0:
            recent_rewards = self.reward_buffers[idx]
            n_obs = len(recent_rewards)
            obs_var = self.var_list[idx]
            
            # Use a fixed prior variance to allow adaptation
            prior_var = 100.0
            
            # Bayesian update using only recent data
            posterior_var = 1.0 / (1.0/prior_var + n_obs/obs_var)
            posterior_mean = posterior_var * (0.0/prior_var + sum(recent_rewards)/obs_var)
            
            self.prior_means[idx] = posterior_mean
            self.prior_vars[idx] = posterior_var
        
        # ==================================== Your Code (End) ====================================

In [ ]:
mab = [Arm(6, 0.5), Arm(4, 0.5)]
regret = [run_sim2(NewThompsonSampling([b.var for b in mab]), mab) for _ in range(50)]
plot(np.mean(regret, axis=0), mab)